# [sec1] Data resize by openCV and write out

In [ ]:
import cv2
import glob
import numpy as np
datapath="/home/stream/Downloads/"
target="ResizeCDD64/"
source="cat_dog_data/"
currentdata="cat_dog_testdata/"
pic_list=glob.glob(datapath+source+currentdata+"*.jpg")

In [ ]:
pic_list[0]

In [ ]:
for k in xrange(len(pic_list)):
    img=cv2.resize(cv2.imread(pic_list[k]),(64,64))
    cv2.imwrite(datapath+target+currentdata+pic_list[k].split('/')[-1], img)

# load datas to numpy type, and write to npy file

outfile_x.npy : images that stores in ndarray with 64*64*3

outfile_y.npy : labels of outfile_x.npy i.e. cat/dog using 0, 1 presents

https://drive.google.com/open?id=0BxRU7n3UdSpqR2YxeDdwVW5uLVE

In [1]:
import cv2
import glob
import numpy as np
datapath="/home/stream/Downloads/"
currentdata="cat_dog_data_all/cat/"
pic_list_cat=glob.glob(datapath+currentdata+"*.jpg")

currentdata="cat_dog_data_all/dog/"
pic_list_dog=glob.glob(datapath+currentdata+"*.jpg")

datarows=len(pic_list_cat)+len(pic_list_dog)
frames = np.empty((datarows, 64,64, 3))
frames2 = np.empty((datarows, 64,64, 3))
y_train=np.empty(datarows)


In [2]:
frames.dtype

dtype('float64')

In [ ]:
#path print
print pic_list_cat[0]
print pic_list_dog[0]

In [ ]:
k=0
print cv2.cvtColor(cv2.resize(cv2.imread(pic_list_cat[k]),(64,64)), cv2.cv.CV_BGR2RGB).shape
print cv2.cvtColor(cv2.resize(cv2.imread(pic_list_cat[k]),(64,64)), cv2.cv.CV_BGR2RGB).dtype

# using Opencv

In [3]:
count=0
#img = cv2.cvtColor(image, cv2.cv.CV_BGR2RGB)

for k in xrange(len(pic_list_cat)):
    frames[k,:,:,:] = cv2.cvtColor(cv2.resize(cv2.imread(pic_list_cat[k]),(64,64)), cv2.cv.CV_BGR2RGB)
y_train[count:count+len(pic_list_cat)]=0
count=count+len(pic_list_cat)

for k in xrange(len(pic_list_dog)):
    frames[count+k,:,:,:] =  cv2.cvtColor(cv2.resize(cv2.imread(pic_list_dog[k]),(64,64)), cv2.cv.CV_BGR2RGB)
y_train[count:count+len(pic_list_dog)]=1



In [4]:
print type(frames[0])
print frames[0].shape
print frames[0].ndim
print frames[0].dtype
print frames[0].max()

<type 'numpy.ndarray'>
(64, 64, 3)
3
float64
255.0


# using skimage

In [ ]:
import skimage
import skimage.io
from skimage.transform import resize

In [ ]:
filename=pic_list_cat[0]
color=True
img = resize(skimage.io.imread(filename, as_grey=not color),(64,64))

In [ ]:
img.shape

In [ ]:
print type(img)
print img.shape
print img.ndim
print img.max()

In [ ]:
count=0

for k in xrange(len(pic_list_cat)):
    frames[k,:,:,:] = skimage.img_as_float(resize(skimage.io.imread(pic_list_cat[k], as_grey=not color),(64,64))).astype(np.float32)
y_train[count:count+len(pic_list_cat)]=0
count=count+len(pic_list_cat)

for k in xrange(len(pic_list_dog)):
    frames[count+k,:,:,:] =  skimage.img_as_float(resize(skimage.io.imread(pic_list_dog[k], as_grey=not color),(64,64))).astype(np.float32)
y_train[count:count+len(pic_list_dog)]=1



In [5]:
len(frames)

25000

In [5]:
np.save("/home/stream/whimh2.0/Bibi/outfile_x",frames)
np.save("/home/stream/whimh2.0/Bibi/outfile_y",y_train)

# [sec2] train a easy logistic regression model

In [ ]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#load preprocessed data
feature_x=np.load("/home/stream/whimh2.0/Bibi/outfile_x.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [ ]:
#create training set and testing set
arr = np.arange(len(feature_x))
np.random.shuffle(arr)
x_train=feature_x[arr]
y_train=y_train[arr]
data={
  'X_train': x_train[:int(len(x_train)*0.8)],
  'y_train': y_train[:int(len(x_train)*0.8)],
  'X_val': x_train[int(len(x_train)*0.8):],
  'y_val': y_train[int(len(x_train)*0.8):],
}
print "there are "+ str(data['X_train'].shape[0]) + " images in training set"
print "there are "+ str(data['X_val'].shape[0]) + " images in testing set"

a Ez tensorflow logistregression

In [ ]:
# create the session first
sess = tf.InteractiveSession()

In [ ]:
# prepare placeholder for X and Y (all these operation will directly place into default graph in sess)
x = tf.placeholder(tf.float32, [None, 64,64,3])
y_tf = tf.placeholder(tf.int32, shape=[None,],name='truth_y')
#for y which is 0,1 needed to be one_hot encoding
y_tf_2=tf.one_hot(y_tf,2)

In [ ]:
# a fc to image and output to 2dim
W = tf.Variable(tf.zeros([64*64*3, 2]))
b = tf.Variable(tf.zeros([2]))

In [ ]:
x_flat = tf.reshape(x, [-1, 64*64*3])
y = tf.nn.softmax(tf.matmul(x_flat, W) + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_tf_2 * tf.log(y), reduction_indices=[1]))
regularizers = (tf.nn.l2_loss(W))
loss=cross_entropy+2e-1*regularizers

In [ ]:
# reduce the loss by applying adam 
train_step = tf.train.AdamOptimizer(1e-6).minimize(loss)

In [ ]:
# a mesurement for accuracy
with tf.name_scope('measure'):
    with tf.name_scope('predict'):
        predctions=tf.argmax(y,1, name='predictions_')
    with tf.name_scope('groundtruth'):
        ground_truth=tf.argmax(y_tf_2,1)
    correct_prediction = tf.equal(predctions, ground_truth)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#initialize all these variable
sess.run(tf.initialize_all_variables())

In [ ]:
k=0
for i in range(10000):
    arr=np.arange(len(data['X_train']))
    np.random.shuffle(arr)
    idx = arr[:50]
    train_step.run(feed_dict={x:data['X_train'][idx], y_tf: data['y_train'][idx]})
    if i%1000==0:
        train_accuracy=sess.run(accuracy, feed_dict={x:data['X_train'], y_tf: data['y_train']})
        testing_accuracy=sess.run(accuracy, feed_dict={x:data['X_val'], y_tf: data['y_val']})
        print("step %d, training accuracy %g, testing acc %g"%(i, train_accuracy,testing_accuracy))

by applying logistic regression can't get a good result.. ~57%

# [sec3] applying a simple CNN model
3x3(WxH) x3(chanel) filter *32 - pooling - 3x3(WxH) x3 *64 - pooling -fc [1024] -dropout -fc [2]-softmax- cross_entropy

In [ ]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

#load preprocessed data
feature_x=np.load("/home/stream/whimh2.0/Bibi/outfile_x.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [ ]:
#create training set and testing set
arr = np.arange(len(feature_x))
np.random.shuffle(arr)
x_train=feature_x[arr]
y_train=y_train[arr]
data={
  'X_train': x_train[:int(len(x_train)*0.8)],
  'y_train': y_train[:int(len(x_train)*0.8)],
  'X_val': x_train[int(len(x_train)*0.8):],
  'y_val': y_train[int(len(x_train)*0.8):],
}
print "there are "+ str(data['X_train'].shape[0]) + " images in training set"
print "there are "+ str(data['X_val'].shape[0]) + " images in testing set"

In [ ]:
#used to create weight
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.0001)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

In [ ]:
#used to perform conv
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


In [ ]:
with tf.name_scope('conv'):
    x_image1 = tf.placeholder(tf.float32, shape=[None, 64,64,3])
    x_image=tf.image.resize_images(x_image1,32,32)
    y_tf = tf.placeholder(tf.int32, shape=[None,],name='truth_y')
    #for y which is 0,1 needed to be one_hot encoding
    y_tf_2=tf.one_hot(y_tf,2)
    keep_prob = tf.placeholder(tf.float32)
    W_conv1 = weight_variable([3,3, 3, 32])
    b_conv1 = bias_variable([32])
    # first conv1
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    W_conv2 = weight_variable([3, 3, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    W_fc1 = weight_variable([8 * 8 * 64, 1024])
    b_fc1 = bias_variable([1024])
with tf.name_scope('Fc'):
    h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    #here a dropout layer
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    #two output
    W_fc2 = weight_variable([1024, 2])
    b_fc2 = bias_variable([2])

    y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    #cross_entropy
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_tf_2 * tf.log(y_conv), reduction_indices=[1]))
    #regularizers
    regularizers = (tf.nn.l2_loss(W_conv2) + tf.nn.l2_loss(W_conv1)+
                    (tf.nn.l2_loss(W_fc1) + tf.nn.l2_loss(W_fc2)))
    regularTerm = tf.placeholder(tf.float32)
    learingrate = tf.placeholder(tf.float32)

    #loss
    loss=cross_entropy+regularTerm*regularizers
    #Adam Optimizer
    train_step = tf.train.AdamOptimizer(learingrate).minimize(loss)
    #measurement
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_tf_2,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
h_pool2
#check the size for pool to fc

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [ ]:
k=0
for i in range(80000):
    arr=np.arange(len(data['X_train']))
    np.random.shuffle(arr)
    idx = arr[:40]
    train_step.run(feed_dict={x_image1:data['X_train'][idx], y_tf: data['y_train'][idx],keep_prob:0.3,regularTerm:1e-4,learingrate:1e-5})
    if i%1000==0:
        train_accuracy=sess.run(accuracy, feed_dict={x_image1:data['X_train'], y_tf: data['y_train'],keep_prob:1})
        testing_accuracy=sess.run(accuracy, feed_dict={x_image1:data['X_val'], y_tf: data['y_val'],keep_prob:1})
        print("step %d, training accuracy %g, testing acc %g"%(i, train_accuracy,testing_accuracy))

In [ ]:
sess.run(cross_entropy, feed_dict={x_image1:data['X_val'], y_tf: data['y_val'],keep_prob:1})

In [ ]:
sess.run(cross_entropy, feed_dict={x_image1:data['X_train'], y_tf: data['y_train'],keep_prob:1})

using all images get a better result => 81.5%~ learning rate 1e-5 ,regularzation 1e-4 

# [sec4] using Pretrained model
https://github.com/ry/tensorflow-vgg16
VGG_ILSVRC_16_layers tfmodel

finetune.npy :pool:5 of all images
can be found in https://drive.google.com/open?id=0BxRU7n3UdSpqR2YxeDdwVW5uLVE

# [sec4-1] forward to pool:5

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [7]:
sess = tf.InteractiveSession()

#load the graph_def from disk
with open("/home/stream/tensorflow-vgg16/vgg16-20160129.tfmodel", mode='rb') as f:
  fileContent = f.read()
graph_def = tf.GraphDef()
graph_def.ParseFromString(fileContent)

In [8]:
# insert graph from vggface16 model
tf.import_graph_def(graph_def,name='vggILSVRC')

In [9]:
#let's check it's form first
train_writer = tf.train.SummaryWriter('/tmp/loser1/train',sess.graph)

just a vgg16
<img src="img/BibiPre1.png",width=200,height=600>

In [10]:
# close it to clear graph
tf.reset_default_graph()
sess.close()

In [11]:
# restart session
sess = tf.InteractiveSession()


In [12]:
#let froward it to pooling5 for all images
x_tf = tf.placeholder(tf.float32, shape=[None, 64,64,3])
#since the original image is 224x224 , we can resize it as the same or smaller(64x64)
#just 32x32 will fails in some layers(since polling/conv).
#x_tf_1=tf.image.resize_images(x_tf,64,64)
x_tf_1=tf.image.resize_images(x_tf,224,224)

y_tf = tf.placeholder(tf.int32, shape=[None,])
# import the graph_def to default graph , replace images with x_tf_1/x_tf
tf.import_graph_def(graph_def, input_map={ "images": x_tf_1 })
print "graph loaded into default graph"

graph = tf.get_default_graph()
#[i.name for i in graph.get_operations()]

graph loaded into default graph


In [13]:
#[i.name for i in graph.get_operations()]

In [14]:
#get the pooling5
feature_net=graph.get_tensor_by_name("import/pool5:0")
#get the pooling4
#feature_net=graph.get_tensor_by_name("import/conv5_2/Relu:0")

In [15]:
feature_net

<tf.Tensor 'import/pool5:0' shape=(?, 7, 7, 512) dtype=float32>

In [16]:
#load preprocessed data
frames=np.load("/home/stream/whimh2.0/Bibi/outfile_x.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [17]:
#it takes ~20GB ram for per 100 image forward
feature_x = sess.run([feature_net], feed_dict={x_tf:frames[:100], y_tf: y_train[:100]})[0]

In [18]:
for i in range(1,250):
    if i%10==0:
        print "current i: "+str(i)
    feature_temp = sess.run([feature_net], feed_dict={x_tf:frames[i*100:(i+1)*100], y_tf: y_train[i*100:(i+1)*100]})[0]
    feature_x=np.concatenate((feature_x, feature_temp), axis=0)

current i: 10
current i: 20
current i: 30
current i: 40
current i: 50
current i: 60
current i: 70
current i: 80
current i: 90
current i: 100
current i: 110
current i: 120
current i: 130
current i: 140
current i: 150
current i: 160
current i: 170
current i: 180
current i: 190
current i: 200
current i: 210
current i: 220
current i: 230
current i: 240


In [19]:
#np.save("/home/stream/whimh2.0/Bibi/finetune_conv5_2",feature_x)
np.save("/home/stream/whimh2.0/Bibi/finetune_vgg_pool5",feature_x)

In [20]:
print feature_x.shape
#clear the graph
tf.reset_default_graph()
sess.close()

(25000, 7, 7, 512)


In [ ]:
import gc
del feature_x
gc.collect()

# [sec4-2] Train the model using pool:5 features

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
feature_x=np.load("/home/stream/whimh2.0/Bibi/finetune_vgg_pool5.npy")
#frames=np.load("/home/stream/whimh2.0/Bibi/outfile_x.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [3]:
#from sklearn.preprocessing import normalize
#norm2 = normalize(feature_x, axis=1)
arr = np.arange(len(feature_x))
np.random.shuffle(arr)
np.save("/home/stream/whimh2.0/Bibi/20161020seed",arr)
x_train=feature_x[arr]
#x_train=norm2[arr]
y_train=y_train[arr]
data={
  'X_train': x_train[:int(len(x_train)*0.8)],
  'y_train': y_train[:int(len(x_train)*0.8)],
  'X_val': x_train[int(len(x_train)*0.8):],
  'y_val': y_train[int(len(x_train)*0.8):],
}
print "there are "+ str(data['X_train'].shape[0]) + " images in training set"
print "there are "+ str(data['X_val'].shape[0]) + " images in testing set"

there are 20000 images in training set
there are 5000 images in testing set


In [4]:
sess = tf.InteractiveSession()

In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.0001)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

In [6]:
#x_tfed = tf.placeholder(tf.float32, shape=[None,2,2,512],name='feature_x')
x_tfed = tf.placeholder(tf.float32, shape=[None,7,7,512],name='feature_x')
y_tf = tf.placeholder(tf.int32, shape=[None,],name='truth_y')

In [7]:
#Here contains the finetune layers
with tf.name_scope('fintune_whimh'):
    keep_prob = tf.placeholder(tf.float32)
    regularTerm = tf.placeholder(tf.float32)
    learningRate = tf.placeholder(tf.float32)

    with tf.name_scope('Fc1'):
        #calculate_entropy
        h_pool3_flat = tf.reshape(x_tfed, [-1, 7*7*512])
        W_whimh_fc1 = weight_variable([7*7*512,1024])
        b_whimh_fc1 = bias_variable([1024])
        h_fc1=tf.nn.relu(tf.matmul(h_pool3_flat, W_whimh_fc1) + b_whimh_fc1) 
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    with tf.name_scope('Fc2'):
        #calculate_entropy
        W_whimh_fc2 = weight_variable([1024, 2])
        b_whimh_fc2 = bias_variable([2])
        y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_whimh_fc2) + b_whimh_fc2, name='predictions_softmax')
        #y_conv = tf.nn.dropout(h_fc2, keep_prob)

#    with tf.name_scope('Fc3'):
#        #calculate_entropy
#        W_whimh_fc3 = weight_variable([1024, 2])
#        b_whimh_fc3 = bias_variable([2])
#        y_conv=tf.nn.softmax(tf.matmul(h_fc2_drop, W_whimh_fc3) + b_whimh_fc3, name='predictions_softmax') 
    y_tf_2=tf.one_hot(y_tf,2)
    with tf.name_scope('loss_calulate'):
        #calculate_entropy
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_tf_2 * tf.log(y_conv), reduction_indices=[1]))
    with tf.name_scope('regular'):    
        regularizers = (tf.nn.l2_loss(W_whimh_fc1) + tf.nn.l2_loss(b_whimh_fc1) +
                        tf.nn.l2_loss(W_whimh_fc2) + tf.nn.l2_loss(b_whimh_fc2) )
                        #tf.nn.l2_loss(W_whimh_fc3) + tf.nn.l2_loss(b_whimh_fc3) )
    with tf.name_scope('loss'):
        #carefully deal overfitting
        loss=cross_entropy+regularTerm*regularizers
    with tf.name_scope('slover'):
        train_step = tf.train.AdamOptimizer(learningRate).minimize(loss)
    with tf.name_scope('measure'):
        with tf.name_scope('predict'):
            predctions=tf.argmax(y_conv,1, name='predictions_')
        with tf.name_scope('groundtruth'):
            ground_truth=tf.argmax(y_tf_2,1)
        correct_prediction = tf.equal(predctions, ground_truth)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
sess.run(tf.initialize_all_variables())

In [9]:
testing_accuracy,testing_cross_entropy = sess.run([accuracy,cross_entropy], feed_dict={   
    x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})

In [10]:
mini_testing_cross_entropy=testing_cross_entropy

In [15]:
print("minimum entropy : %g"%mini_testing_cross_entropy)

minimum entropy : 0.141104


In [14]:
k=0
WeiDecayCount=0
gamma=0.08
BaseLearningR=1e-7
#maxinterval=1
#interval=0

learningR=BaseLearningR
for i in range(1,10000):
  arr=np.arange(len(data['X_train']))
  np.random.shuffle(arr)
  idx = arr[:30]
  train_step.run(feed_dict={x_tfed:data['X_train'][idx], y_tf: data['y_train'][idx],keep_prob:0.5,learningRate:learningR,regularTerm:2e-3})
  
  if i%100==0:
    print("measurement")
    #interval+=1
    ############testing acc#########
    testing_accuracy,testing_cross_entropy = sess.run([accuracy,cross_entropy], feed_dict={   
        x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})
    ############ Wegiht Decay #########
    #WeiDecayCount+=1
    learningR=BaseLearningR*np.exp(-gamma*WeiDecayCount)
    print("==========Weight have been decaied %g=============="%learningR)

    if(mini_testing_cross_entropy>testing_cross_entropy):
        #interval=0
        mini_testing_cross_entropy=testing_cross_entropy
        saver=tf.train.Saver(tf.all_variables())
        saver.save(sess,"vgg_pool5.ckpt")
        print("**************minimum entropy updated %g **************"%mini_testing_cross_entropy)
    ############training acc###########
    if i%500==0:
        train_accuracy = sess.run(accuracy, feed_dict={   
                x_tfed:data['X_train'], y_tf: data['y_train'],keep_prob:1})
        print("step %d, training accuracy %g, testing acc %g \n,=== cross_entropy %g ==="%(i, train_accuracy,testing_accuracy,testing_cross_entropy))
    else:
        print("step %d, testing acc %g, cross_entropy %g"%(i,testing_accuracy,testing_cross_entropy))
        
#overall
print("test accuracy %g"%accuracy.eval(feed_dict={
    x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1}))

measurement
==========Weight have been decaied 1e-07==============
step 100, testing acc 0.884, cross_entropy 0.482654
measurement
==========Weight have been decaied 1e-07==============
step 200, testing acc 0.9028, cross_entropy 0.345979
measurement
==========Weight have been decaied 1e-07==============
step 300, testing acc 0.9092, cross_entropy 0.277073
measurement
==========Weight have been decaied 1e-07==============
step 400, testing acc 0.9178, cross_entropy 0.238784
measurement
==========Weight have been decaied 1e-07==============
step 500, training accuracy 0.9325, testing acc 0.9208 
,=== cross_entropy 0.215044 ===
measurement
==========Weight have been decaied 1e-07==============
step 600, testing acc 0.9236, cross_entropy 0.199973
measurement
==========Weight have been decaied 1e-07==============
step 700, testing acc 0.926, cross_entropy 0.189663
measurement
==========Weight have been decaied 1e-07==============
step 800, testing acc 0.9256, cross_entropy 0.183156
measure

In [21]:
#using [7*7*512 x 1024]-[1024 x2] two Fc can achieve ~0.945 accuracy 
#cross_entropy is about ~0.15
sess.run(cross_entropy, feed_dict={x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})


0.13136297

In [22]:
saver=tf.train.Saver(tf.all_variables())
saver.save(sess,"vggface16.ckpt")

'vggface16.ckpt'

In [24]:
tf.reset_default_graph()
sess.close()

# [sec5]Tune from resnet

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
sess = tf.InteractiveSession()
graph = tf.get_default_graph()

In [2]:
#load preprocessed data
feature_x=np.load("/home/stream/whimh2.0/Bibi/outfile_x.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [3]:
x_tf = tf.placeholder(tf.float32, shape=[None, 64,64,3])
#since the original image is 224x224 , we can resize it as the same or smaller(64x64)
#just 32x32 will fails in some layers(since polling/conv).
#x_tf_1=tf.image.resize_images(x_tf,64,64)
x_tf_1=tf.image.resize_images(x_tf,224,224)


In [ ]:
frames=sess.run(x_tf_1, feed_dict={x_tf:(feature_x)})

In [ ]:
del feature_x

In [ ]:
newsaver=tf.train.import_meta_graph('/home/stream/tensorflow-resnet-pretrained-20160509/ResNet-L50.meta')
newsaver.restore(sess,'/home/stream/tensorflow-resnet-pretrained-20160509/ResNet-L50.ckpt')

In [ ]:
#save the current graph and see what happens
train_writer = tf.train.SummaryWriter('/tmp/resnet/train',sess.graph)

In [ ]:
graph = tf.get_default_graph()
#[i.name for i in graph.get_operations()]

In [ ]:
feature_net=graph.get_tensor_by_name("scale5/block3/Relu:0")
#feature_net=graph.get_tensor_by_name("mul:0")

In [ ]:
images=graph.get_tensor_by_name("images:0")

In [ ]:
# take a look at this
print images
print feature_net
print frames.shape

In [ ]:
feature_x=sess.run(feature_net, feed_dict={images:(frames[:1000]/255)})

In [ ]:
print feature_x.shape

In [ ]:
%time
for i in range(1,25):
    if i%5==0:
        print "current i: "+str(i)
    feature_temp = sess.run([feature_net], feed_dict={images:frames[i*1000:(i+1)*1000]/255})[0]
    feature_x=np.concatenate((feature_x, feature_temp), axis=0)

In [ ]:
np.save("/home/stream/whimh2.0/Bibi/finetune_res_50_scale5",feature_x)

# [sec5-1]Tune from resnet-1

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [3]:
feature_x=np.load("/home/stream/whimh2.0/Bibi/finetune_res_50_scale5.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [4]:
print feature_x.shape

(25000, 7, 7, 2048)


In [5]:
#arr = np.arange(len(feature_x))
#np.random.shuffle(arr)
arr=np.load("/home/stream/whimh2.0/Bibi/20161020seed.npy")

x_train=feature_x[arr]
y_train=y_train[arr]
data={
  'X_train': x_train[:int(len(x_train)*0.8)],
  'y_train': y_train[:int(len(x_train)*0.8)],
  'X_val': x_train[int(len(x_train)*0.8):],
  'y_val': y_train[int(len(x_train)*0.8):],
}
print "there are "+ str(data['X_train'].shape[0]) + " images in training set"
print "there are "+ str(data['X_val'].shape[0]) + " images in testing set"

there are 20000 images in training set
there are 5000 images in testing set


In [6]:
import gc
del feature_x
gc.collect()

12

In [7]:
sess = tf.InteractiveSession()

In [8]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.0001)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

In [9]:
x_tfed = tf.placeholder(tf.float32, shape=[None,7,7,2048],name='feature_x')
y_tf = tf.placeholder(tf.int32, shape=[None,],name='truth_y')

In [10]:
#Here contains the finetune layers
with tf.name_scope('fintune_whimh'):
    keep_prob = tf.placeholder(tf.float32)
    regularTerm = tf.placeholder(tf.float32)
    learningRate = tf.placeholder(tf.float32)

    with tf.name_scope('Fc1'):
        #calculate_entropy
        h_pool3_flat = tf.reshape(x_tfed, [-1, 7*7*2048])
        W_whimh_fc1 = weight_variable([7*7*2048,1024])
        b_whimh_fc1 = bias_variable([1024])
        h_fc1=tf.nn.relu(tf.matmul(h_pool3_flat, W_whimh_fc1) + b_whimh_fc1) 
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    with tf.name_scope('Fc2'):
        #calculate_entropy
        W_whimh_fc2 = weight_variable([1024, 2])
        b_whimh_fc2 = bias_variable([2])
        y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_whimh_fc2) + b_whimh_fc2, name='predictions_softmax')
        #y_conv = tf.nn.dropout(h_fc2, keep_prob)

#    with tf.name_scope('Fc3'):
#        #calculate_entropy
#        W_whimh_fc3 = weight_variable([1024, 2])
#        b_whimh_fc3 = bias_variable([2])
#        y_conv=tf.nn.softmax(tf.matmul(h_fc2_drop, W_whimh_fc3) + b_whimh_fc3, name='predictions_softmax') 
    y_tf_2=tf.one_hot(y_tf,2)
    with tf.name_scope('loss_calulate'):
        #calculate_entropy
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_tf_2 * tf.log(y_conv), reduction_indices=[1]))
    with tf.name_scope('regular'):    
        regularizers = (tf.nn.l2_loss(W_whimh_fc1) + tf.nn.l2_loss(b_whimh_fc1) +
                        tf.nn.l2_loss(W_whimh_fc2) + tf.nn.l2_loss(b_whimh_fc2) )
                        #tf.nn.l2_loss(W_whimh_fc3) + tf.nn.l2_loss(b_whimh_fc3) )
    with tf.name_scope('loss'):
        #carefully deal overfitting
        loss=cross_entropy+regularTerm*regularizers
    with tf.name_scope('slover'):
        train_step = tf.train.AdamOptimizer(learningRate).minimize(loss)
    with tf.name_scope('measure'):
        with tf.name_scope('predict'):
            predctions=tf.argmax(y_conv,1, name='predictions_')
        with tf.name_scope('groundtruth'):
            ground_truth=tf.argmax(y_tf_2,1)
        correct_prediction = tf.equal(predctions, ground_truth)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
sess.run(tf.initialize_all_variables())

In [12]:
testing_accuracy,testing_cross_entropy = sess.run([accuracy,cross_entropy], feed_dict={   
    x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})

In [13]:
mini_testing_cross_entropy=testing_cross_entropy

In [14]:
print("minimum entropy : %g"%mini_testing_cross_entropy)

minimum entropy : 0.693145


In [16]:
k=0
WeiDecayCount=0
gamma=0.02
BaseLearningR=1.7e-7
#maxinterval=1
#interval=0

learningR=BaseLearningR
for i in range(1,10000):
  arr=np.arange(len(data['X_train']))
  np.random.shuffle(arr)
  idx = arr[:30]
  train_step.run(feed_dict={x_tfed:data['X_train'][idx], y_tf: data['y_train'][idx],keep_prob:0.5,learningRate:learningR,regularTerm:1e-3})
  
  if i%100==0:
    print("measurement")
    #interval+=1
    ############testing acc#########
    testing_accuracy,testing_cross_entropy = sess.run([accuracy,cross_entropy], feed_dict={   
        x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})
    ############ Wegiht Decay #########
    WeiDecayCount+=1
    learningR=BaseLearningR*np.exp(-gamma*WeiDecayCount)
    print("==========Weight have been decaied %g=============="%learningR)

    if(mini_testing_cross_entropy>testing_cross_entropy):
        #interval=0
        mini_testing_cross_entropy=testing_cross_entropy
        saver=tf.train.Saver(tf.all_variables())
        saver.save(sess,"resnet50.ckpt")
        print("**************minimum entropy updated %g **************"%mini_testing_cross_entropy)
    ############training acc###########
    if i%1000==0:
        train_accuracy = sess.run(accuracy, feed_dict={   
                x_tfed:data['X_train'], y_tf: data['y_train'],keep_prob:1})
        print("step %d, training accuracy %g, testing acc %g \n,=== cross_entropy %g ==="%(i, train_accuracy,testing_accuracy,testing_cross_entropy))
    else:
        print("step %d, testing acc %g, cross_entropy %g"%(i,testing_accuracy,testing_cross_entropy))
        
#overall
print("test accuracy %g"%accuracy.eval(feed_dict={
    x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1}))

measurement
==========Weight have been decaied 1.66634e-07==============
step 100, testing acc 0.9656, cross_entropy 0.0804957
measurement
==========Weight have been decaied 1.63334e-07==============
step 200, testing acc 0.9664, cross_entropy 0.0804013
measurement
==========Weight have been decaied 1.601e-07==============
step 300, testing acc 0.967, cross_entropy 0.080311
measurement
==========Weight have been decaied 1.5693e-07==============
step 400, testing acc 0.9668, cross_entropy 0.0803166
measurement
==========Weight have been decaied 1.53822e-07==============
step 500, testing acc 0.9668, cross_entropy 0.080424
measurement
==========Weight have been decaied 1.50776e-07==============
step 600, testing acc 0.9666, cross_entropy 0.0804444
measurement
==========Weight have been decaied 1.47791e-07==============
step 700, testing acc 0.9658, cross_entropy 0.0805089
measurement
==========Weight have been decaied 1.44864e-07==============
step 800, testing acc 0.9666, cross_entropy 

KeyboardInterrupt: 

In [15]:
sess.run(cross_entropy, feed_dict={x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})

0.072975889

In [19]:
saver=tf.train.Saver(tf.all_variables())
saver.save(sess,"resnet50.ckpt")

'resnet50.ckpt'

In [26]:
tf.reset_default_graph()
sess.close()

In [ ]:
# can achieve 96.8~

# [sec5-2]Tune from resnet 100

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
sess = tf.InteractiveSession()
graph = tf.get_default_graph()

In [ ]:
#load preprocessed data
feature_x=np.load("/home/stream/whimh2.0/Bibi/outfile_x.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [ ]:
x_tf = tf.placeholder(tf.float32, shape=[None, 64,64,3])
#since the original image is 224x224 , we can resize it as the same or smaller(64x64)
#just 32x32 will fails in some layers(since polling/conv).
#x_tf_1=tf.image.resize_images(x_tf,64,64)
x_tf_1=tf.image.resize_images(x_tf,224,224)


In [ ]:
frames=sess.run(x_tf_1, feed_dict={x_tf:(feature_x)})

In [ ]:
del feature_x

In [ ]:
newsaver=tf.train.import_meta_graph('/home/stream/tensorflow-resnet-pretrained-20160509/ResNet-L101.meta')
newsaver.restore(sess,'/home/stream/tensorflow-resnet-pretrained-20160509/ResNet-L101.ckpt')

In [ ]:
#save the current graph and see what happens
train_writer = tf.train.SummaryWriter('/tmp/resnet101/train',sess.graph)

In [ ]:
graph = tf.get_default_graph()
#[i.name for i in graph.get_operations()]

In [ ]:
feature_net=graph.get_tensor_by_name("scale5/block3/Relu:0")
#feature_net=graph.get_tensor_by_name("mul:0")

In [ ]:
images=graph.get_tensor_by_name("images:0")

In [ ]:
# take a look at this
print images
print feature_net
print frames.shape

In [ ]:
feature_x=sess.run(feature_net, feed_dict={images:(frames[:1000]/255)})

In [ ]:
print feature_x.shape

In [ ]:
%time
for i in range(1,25):
    if i%5==0:
        print "current i: "+str(i)
    feature_temp = sess.run([feature_net], feed_dict={images:frames[i*1000:(i+1)*1000]/255})[0]
    feature_x=np.concatenate((feature_x, feature_temp), axis=0)

In [ ]:
np.save("/home/stream/whimh2.0/Bibi/finetune_res_101_scale5",feature_x)

# [sec 5-3] fine tune resnet101


In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
feature_x=np.load("/home/stream/whimh2.0/Bibi/finetune_res_101_scale5.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [3]:
y_train


array([0, 0, 0, ..., 1, 1, 1])

In [4]:
print feature_x.shape

(25000, 7, 7, 2048)


In [5]:
#arr = np.arange(len(feature_x))
#np.random.shuffle(arr)
arr=np.load("/home/stream/whimh2.0/Bibi/20161020seed.npy")

x_train=feature_x[arr]
y_train=y_train[arr]
data={
  'X_train': x_train[:int(len(x_train)*0.8)],
  'y_train': y_train[:int(len(x_train)*0.8)],
  'X_val': x_train[int(len(x_train)*0.8):],
  'y_val': y_train[int(len(x_train)*0.8):],
}
print "there are "+ str(data['X_train'].shape[0]) + " images in training set"
print "there are "+ str(data['X_val'].shape[0]) + " images in testing set"

there are 20000 images in training set
there are 5000 images in testing set


In [6]:
import gc
del feature_x
gc.collect()

4

In [7]:
sess = tf.InteractiveSession()

In [8]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.0001)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

In [9]:
x_tfed = tf.placeholder(tf.float32, shape=[None,7,7,2048],name='feature_x')
y_tf = tf.placeholder(tf.int32, shape=[None,],name='truth_y')

In [10]:
#Here contains the finetune layers
with tf.name_scope('fintune_whimh'):
    keep_prob = tf.placeholder(tf.float32)
    regularTerm = tf.placeholder(tf.float32)
    learningRate = tf.placeholder(tf.float32)

    with tf.name_scope('Fc1'):
        #calculate_entropy
        h_pool3_flat = tf.reshape(x_tfed, [-1, 7*7*2048])
        W_whimh_fc1 = weight_variable([7*7*2048,1024])
        b_whimh_fc1 = bias_variable([1024])
        h_fc1=tf.nn.relu(tf.matmul(h_pool3_flat, W_whimh_fc1) + b_whimh_fc1) 
        #y_conv=tf.matmul(h_pool3_flat, W_whimh_fc1) + b_whimh_fc1
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    with tf.name_scope('Fc2'):
        #calculate_entropy
        W_whimh_fc2 = weight_variable([1024, 2])
        b_whimh_fc2 = bias_variable([2])
        y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_whimh_fc2) + b_whimh_fc2, name='predictions_softmax')
#        h_fc2=tf.nn.relu(tf.matmul(h_fc1_drop, W_whimh_fc2) + b_whimh_fc2) 
#        h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)
#    with tf.name_scope('Fc3'):
        #calculate_entropy
#        W_whimh_fc3 = weight_variable([512, 2])
#        b_whimh_fc3 = bias_variable([2])
#        y_conv=tf.nn.softmax(tf.matmul(h_fc2_drop, W_whimh_fc3) + b_whimh_fc3, name='predictions_softmax') 
    y_tf_2=tf.one_hot(y_tf,2)
    with tf.name_scope('loss_calulate'):
        #calculate_entropy
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_tf_2 * tf.log(y_conv), reduction_indices=[1]))
    with tf.name_scope('regular'):    
        regularizers = (tf.nn.l2_loss(W_whimh_fc1) + tf.nn.l2_loss(b_whimh_fc1) +
                        tf.nn.l2_loss(W_whimh_fc2) + tf.nn.l2_loss(b_whimh_fc2) )
                        #tf.nn.l2_loss(W_whimh_fc3) + tf.nn.l2_loss(b_whimh_fc3) )
    with tf.name_scope('loss'):
        #carefully deal overfitting
        loss=cross_entropy+regularTerm*regularizers
    with tf.name_scope('slover'):
        train_step = tf.train.AdamOptimizer(learningRate).minimize(loss)
    with tf.name_scope('measure'):
        with tf.name_scope('predict'):
            predctions=tf.argmax(y_conv,1, name='predictions_')
        with tf.name_scope('groundtruth'):
            ground_truth=tf.argmax(y_tf_2,1)
        correct_prediction = tf.equal(predctions, ground_truth)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
sess.run(tf.initialize_all_variables())

In [12]:
testing_accuracy,testing_cross_entropy = sess.run([accuracy,cross_entropy], feed_dict={   
    x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})

In [13]:
mini_testing_cross_entropy=testing_cross_entropy

In [14]:
print("minimum entropy : %g"%mini_testing_cross_entropy)

minimum entropy : 0.693147


In [15]:
k=0
WeiDecayCount=0
gamma=0.03
BaseLearningR=5e-6
#maxinterval=1
#interval=0

learningR=BaseLearningR
for i in range(1,10000):
  arr=np.arange(len(data['X_train']))
  np.random.shuffle(arr)
  idx = arr[:30]
  train_step.run(feed_dict={x_tfed:data['X_train'][idx], y_tf: data['y_train'][idx],keep_prob:0.5,learningRate:learningR,regularTerm:2e-3})
  
  if i%100==0:
    print("measurement")
    #interval+=1
    ############testing acc#########
    testing_accuracy,testing_cross_entropy = sess.run([accuracy,cross_entropy], feed_dict={   
        x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})
    ############ Wegiht Decay #########
    WeiDecayCount+=1
    learningR=BaseLearningR*np.exp(-gamma*WeiDecayCount)
    print("==========Weight have been decaied %g=============="%learningR)

    if(mini_testing_cross_entropy>testing_cross_entropy):
        #interval=0
        mini_testing_cross_entropy=testing_cross_entropy
        saver=tf.train.Saver(tf.all_variables())
        saver.save(sess,"resnet101.ckpt")
        print("**************minimum entropy updated %g **************"%mini_testing_cross_entropy)
    ############training acc###########
    if i%1000==0:
        train_accuracy = sess.run(accuracy, feed_dict={   
                x_tfed:data['X_train'], y_tf: data['y_train'],keep_prob:1})
        print("step %d, training accuracy %g, testing acc %g \n,=== cross_entropy %g ==="%(i, train_accuracy,testing_accuracy,testing_cross_entropy))
    else:
        print("step %d, testing acc %g, cross_entropy %g"%(i,testing_accuracy,testing_cross_entropy))
        
#overall
print("test accuracy %g"%accuracy.eval(feed_dict={
    x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1}))

measurement
==========Weight have been decaied 4.85223e-06==============
**************minimum entropy updated 0.196437 **************
step 100, testing acc 0.9612, cross_entropy 0.196437
measurement
==========Weight have been decaied 4.70882e-06==============
**************minimum entropy updated 0.110937 **************
step 200, testing acc 0.9664, cross_entropy 0.110937
measurement
==========Weight have been decaied 4.56966e-06==============
**************minimum entropy updated 0.0943852 **************
step 300, testing acc 0.9666, cross_entropy 0.0943852
measurement
==========Weight have been decaied 4.4346e-06==============
**************minimum entropy updated 0.0852987 **************
step 400, testing acc 0.968, cross_entropy 0.0852987
measurement
==========Weight have been decaied 4.30354e-06==============
**************minimum entropy updated 0.0806429 **************
step 500, testing acc 0.9706, cross_entropy 0.0806429
measurement
==========Weight have been decaied 4.17635e-

KeyboardInterrupt: 

In [ ]:
i

In [22]:
sess.run(cross_entropy, feed_dict={x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})


0.089114882

In [16]:
print("minimum entropy : %g"%mini_testing_cross_entropy)
#resnet F1,F2,F3 can achieve 97.4~
#resnet F1,F2 can acheive 97.46~

minimum entropy : 0.0689698


# sec[5-5] Resnet 151

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
sess = tf.InteractiveSession()
graph = tf.get_default_graph()

In [ ]:
#load preprocessed data
feature_x=np.load("/home/stream/whimh2.0/Bibi/outfile_x.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [ ]:
x_tf = tf.placeholder(tf.float32, shape=[None, 64,64,3])
#since the original image is 224x224 , we can resize it as the same or smaller(64x64)
#just 32x32 will fails in some layers(since polling/conv).
#x_tf_1=tf.image.resize_images(x_tf,64,64)
x_tf_1=tf.image.resize_images(x_tf,224,224)


In [ ]:
frames=sess.run(x_tf_1, feed_dict={x_tf:(feature_x)})

In [ ]:
del feature_x

In [ ]:
newsaver=tf.train.import_meta_graph('/home/stream/tensorflow-resnet-pretrained-20160509/ResNet-L152.meta')
newsaver.restore(sess,'/home/stream/tensorflow-resnet-pretrained-20160509/ResNet-L152.ckpt')

In [ ]:
#save the current graph and see what happens
train_writer = tf.train.SummaryWriter('/tmp/resnet152/train',sess.graph)

In [ ]:
graph = tf.get_default_graph()
#[i.name for i in graph.get_operations()]

In [ ]:
feature_net=graph.get_tensor_by_name("scale5/block3/Relu:0")
#feature_net=graph.get_tensor_by_name("mul:0")

In [ ]:
images=graph.get_tensor_by_name("images:0")

In [ ]:
# take a look at this
print images
print feature_net
print frames.shape

In [ ]:
feature_x=sess.run(feature_net, feed_dict={images:(frames[:1000]/255)})

In [ ]:
print feature_x.shape

In [ ]:
%time
for i in range(1,25):
    if i%5==0:
        print "current i: "+str(i)
    feature_temp = sess.run([feature_net], feed_dict={images:frames[i*1000:(i+1)*1000]/255})[0]
    feature_x=np.concatenate((feature_x, feature_temp), axis=0)

In [ ]:
np.save("/home/stream/whimh2.0/Bibi/finetune_res_152_scale5",feature_x)

# [sec 5-4] fine tune resnet151

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
np.exp(-6*0.05)

0.74081822068171788

In [3]:
feature_x=np.load("/home/stream/whimh2.0/Bibi/finetune_res_152_scale5.npy")
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [4]:
y_train

array([0, 0, 0, ..., 1, 1, 1])

In [5]:
print feature_x.shape

(25000, 7, 7, 2048)


In [6]:
#arr = np.arange(len(feature_x))
#np.random.shuffle(arr)
arr=np.load("/home/stream/whimh2.0/Bibi/20161020seed.npy")
x_train=feature_x[arr]
y_train=y_train[arr]
data={
  'X_train': x_train[:int(len(x_train)*0.8)],
  'y_train': y_train[:int(len(x_train)*0.8)],
  'X_val': x_train[int(len(x_train)*0.8):],
  'y_val': y_train[int(len(x_train)*0.8):],
}
print "there are "+ str(data['X_train'].shape) + " images in training set"
print "there are "+ str(data['X_val'].shape) + " images in testing set"

there are (20000, 7, 7, 2048) images in training set
there are (5000, 7, 7, 2048) images in testing set


In [7]:
import gc
del feature_x
gc.collect()

12

In [8]:
sess = tf.InteractiveSession()

In [9]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.0001)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

In [10]:
x_tfed = tf.placeholder(tf.float32, shape=[None,7,7,2048],name='feature_x')
y_tf = tf.placeholder(tf.int32, shape=[None,],name='truth_y')

In [11]:
#Here contains the finetune layers
with tf.name_scope('fintune_whimh'):
    keep_prob = tf.placeholder(tf.float32)
    regularTerm = tf.placeholder(tf.float32)
    learningRate = tf.placeholder(tf.float32)

    with tf.name_scope('Fc1'):
        #calculate_entropy
        h_pool3_flat = tf.reshape(x_tfed, [-1, 7*7*2048])
        W_whimh_fc1 = weight_variable([7*7*2048,1024])
        b_whimh_fc1 = bias_variable([1024])
        h_fc1=tf.nn.relu(tf.matmul(h_pool3_flat, W_whimh_fc1) + b_whimh_fc1) 
        #y_conv=tf.matmul(h_pool3_flat, W_whimh_fc1) + b_whimh_fc1
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    with tf.name_scope('Fc2'):
        #calculate_entropy
        W_whimh_fc2 = weight_variable([1024, 2])
        b_whimh_fc2 = bias_variable([2])
        y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_whimh_fc2) + b_whimh_fc2, name='predictions_softmax')
#        h_fc2=tf.nn.relu(tf.matmul(h_fc1_drop, W_whimh_fc2) + b_whimh_fc2) 
#        h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)
#    with tf.name_scope('Fc3'):
        #calculate_entropy
#        W_whimh_fc3 = weight_variable([512, 2])
#        b_whimh_fc3 = bias_variable([2])
#        y_conv=tf.nn.softmax(tf.matmul(h_fc2_drop, W_whimh_fc3) + b_whimh_fc3, name='predictions_softmax') 
    y_tf_2=tf.one_hot(y_tf,2)
    with tf.name_scope('loss_calulate'):
        #calculate_entropy
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_tf_2 * tf.log(y_conv), reduction_indices=[1]))
    with tf.name_scope('regular'):    
        regularizers = (tf.nn.l2_loss(W_whimh_fc1) + tf.nn.l2_loss(b_whimh_fc1) +
                        tf.nn.l2_loss(W_whimh_fc2) + tf.nn.l2_loss(b_whimh_fc2) )
                        #tf.nn.l2_loss(W_whimh_fc3) + tf.nn.l2_loss(b_whimh_fc3) )
    with tf.name_scope('loss'):
        #carefully deal overfitting
        loss=cross_entropy+regularTerm*regularizers
    with tf.name_scope('slover'):
        train_step = tf.train.AdamOptimizer(learningRate).minimize(loss)
    with tf.name_scope('measure'):
        with tf.name_scope('predict'):
            predctions=tf.argmax(y_conv,1, name='predictions_')
        with tf.name_scope('groundtruth'):
            ground_truth=tf.argmax(y_tf_2,1)
        correct_prediction = tf.equal(predctions, ground_truth)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
sess.run(tf.initialize_all_variables())

In [13]:
testing_accuracy,testing_cross_entropy = sess.run([accuracy,cross_entropy], feed_dict={   
    x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})

In [14]:
mini_testing_cross_entropy=testing_cross_entropy

In [15]:
print("minimum entropy : %g"%mini_testing_cross_entropy)

minimum entropy : 0.693155


In [ ]:
k=0
WeiDecayCount=0
gamma=0.03
BaseLearningR=5e-6
#maxinterval=1
#interval=0

learningR=BaseLearningR
for i in range(1,10000):
  arr=np.arange(len(data['X_train']))
  np.random.shuffle(arr)
  idx = arr[:30]
  train_step.run(feed_dict={x_tfed:data['X_train'][idx], y_tf: data['y_train'][idx],keep_prob:0.5,learningRate:learningR,regularTerm:2e-3})
  
  if i%100==0:
    print("measurement")
    #interval+=1
    ############testing acc#########
    testing_accuracy,testing_cross_entropy = sess.run([accuracy,cross_entropy], feed_dict={   
        x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})
    ############ Wegiht Decay #########
    WeiDecayCount+=1
    learningR=BaseLearningR*np.exp(-gamma*WeiDecayCount)
    print("==========Weight have been decaied %g=============="%learningR)

    if(mini_testing_cross_entropy>testing_cross_entropy):
        #interval=0
        mini_testing_cross_entropy=testing_cross_entropy
        saver=tf.train.Saver(tf.all_variables())
        saver.save(sess,"resnet152.ckpt")
        print("**************minimum entropy updated %g **************"%mini_testing_cross_entropy)
    ############training acc###########
    if i%1000==0:
        train_accuracy = sess.run(accuracy, feed_dict={   
                x_tfed:data['X_train'], y_tf: data['y_train'],keep_prob:1})
        print("step %d, training accuracy %g, testing acc %g \n,=== cross_entropy %g ==="%(i, train_accuracy,testing_accuracy,testing_cross_entropy))
    else:
        print("step %d, testing acc %g, cross_entropy %g"%(i,testing_accuracy,testing_cross_entropy))
        
#overall
print("test accuracy %g"%accuracy.eval(feed_dict={
    x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1}))

measurement
==========Weight have been decaied 4.85223e-06==============
**************minimum entropy updated 0.184972 **************
step 100, testing acc 0.9574, cross_entropy 0.184972


In [ ]:
print("minimum entropy : %g"%mini_testing_cross_entropy)

In [ ]:
i

In [13]:
sess.run(cross_entropy, feed_dict={x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})

0.13115662

In [43]:
saver=tf.train.Saver(tf.all_variables())
saver.save(sess,"Resnet152.ckpt")
#resnet F1,F2 can acheive 97.62~
#cross entroy ~0.06

'Resnet152.ckpt'

In [18]:
# close it to clear graph
tf.reset_default_graph()
sess.close()

# Test_set_Reading

In [ ]:

import glob
import numpy as np
import cv2
datapath="/home/stream/Downloads/cat_dog_data_all/test/"

pic_list=glob.glob(datapath+"*.jpg")

In [ ]:
w=[int(i.split('/')[-1].split('.')[0]) for i in pic_list]

In [ ]:
frames = np.empty((len(pic_list), 64,64, 3))

In [ ]:

for k in xrange(len(pic_list)):
    frames[k,:,:,:] = cv2.cvtColor(cv2.resize(cv2.imread(pic_list[k]),(64,64)), cv2.cv.CV_BGR2RGB)


In [ ]:
assert len(frames)==len(w)

In [ ]:
np.save("/home/stream/whimh2.0/Bibi/test_ID",w)
np.save("/home/stream/whimh2.0/Bibi/test_X",frames)

In [ ]:
len(w)


# restore sess

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
newsaver=tf.train.import_meta_graph('/home/stream/whimh2.0/Bibi/trained_model/Resnet152.ckpt.meta')
newsaver.restore(sess,'/home/stream/whimh2.0/Bibi/trained_model/Resnet152.ckpt')

In [ ]:
feature_x=np.load("/home/stream/whimh2.0/Bibi/test_res_152_scale5.npy")

In [ ]:
y_ID=np.load("/home/stream/whimh2.0/Bibi/test_ID.npy")

In [ ]:
y_ID[:5]
#dog,cat,cat,dog,dog
#1,0,0,1,1

In [ ]:
#[i.name for i in graph.get_operations()]
graph = tf.get_default_graph()
predctions=graph.get_tensor_by_name("fintune_whimh/measure/predict/predictions_:0")
x_tfed=graph.get_tensor_by_name("feature_x:0")
keep_prob=graph.get_tensor_by_name("fintune_whimh/Placeholder:0")

In [ ]:
print keep_prob
print x_tfed
print predctions

In [ ]:
w=sess.run(predctions, feed_dict={x_tfed:feature_x[:5],keep_prob:1.0})

In [ ]:
print w

In [ ]:
sess.close()

# [sec6] mixture all features

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
feature_x=np.empty((100352*2,25000),dtype=np.float32)

In [3]:
feature_x[:100352]=np.load("/home/stream/whimh2.0/Bibi/finetune_conv5_2.npy").reshape(25000,100352).transpose(1,0)
feature_x[100352:100352*2]=np.load("/home/stream/whimh2.0/Bibi/finetune_res_152_scale5.npy").reshape(25000,100352).transpose(1,0)

#feature_50=np.load("/home/stream/whimh2.0/Bibi/finetune_res_50_scale5.npy")
#feature_152=np.load("/home/stream/whimh2.0/Bibi/finetune_res_152_scale5.npy")


In [5]:
feature_x=feature_x.transpose(1,0)

In [6]:
y_train=np.load("/home/stream/whimh2.0/Bibi/outfile_y.npy").astype(int)

In [8]:
print y_train.shape

(25000,)


In [9]:
arr = np.arange(len(feature_x))
np.random.shuffle(arr)
x_train=feature_x[arr]
y_train=y_train[arr]
data={
  'X_train': x_train[:int(len(x_train)*0.8)],
  'y_train': y_train[:int(len(x_train)*0.8)],
  'X_val': x_train[int(len(x_train)*0.8):],
  'y_val': y_train[int(len(x_train)*0.8):],
}
print "there are "+ str(data['X_train'].shape) + " images in training set"
print "there are "+ str(data['X_val'].shape) + " images in testing set"

there are (20000, 200704) images in training set
there are (5000, 200704) images in testing set


In [10]:
import gc
del feature_x
gc.collect()

0

In [18]:
sess = tf.InteractiveSession()

In [19]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.0001)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

In [20]:
x_tfed = tf.placeholder(tf.float32, shape=[None,200704],name='feature_x')
y_tf = tf.placeholder(tf.int32, shape=[None,],name='truth_y')

In [22]:
#Here contains the finetune layers
with tf.name_scope('fintune_whimh'):
    keep_prob = tf.placeholder(tf.float32)
    regularTerm = tf.placeholder(tf.float32)
    learningRate = tf.placeholder(tf.float32)

    with tf.name_scope('Fc1'):
        #calculate_entropy
        W_whimh_fc1 = weight_variable([7*7*2048*2,512])
        b_whimh_fc1 = bias_variable([512])
        h_fc1=tf.nn.relu(tf.matmul(x_tfed, W_whimh_fc1) + b_whimh_fc1) 
        #y_conv=tf.matmul(h_pool3_flat, W_whimh_fc1) + b_whimh_fc1
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    with tf.name_scope('Fc2'):
        #calculate_entropy
        W_whimh_fc2 = weight_variable([512, 2])
        b_whimh_fc2 = bias_variable([2])
        y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_whimh_fc2) + b_whimh_fc2, name='predictions_softmax')
#        h_fc2=tf.nn.relu(tf.matmul(h_fc1_drop, W_whimh_fc2) + b_whimh_fc2) 
#        h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)
#    with tf.name_scope('Fc3'):
        #calculate_entropy
#        W_whimh_fc3 = weight_variable([512, 2])
#        b_whimh_fc3 = bias_variable([2])
#        y_conv=tf.nn.softmax(tf.matmul(h_fc2_drop, W_whimh_fc3) + b_whimh_fc3, name='predictions_softmax') 
    y_tf_2=tf.one_hot(y_tf,2)
    with tf.name_scope('loss_calulate'):
        #calculate_entropy
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_tf_2 * tf.log(y_conv), reduction_indices=[1]))
    with tf.name_scope('regular'):    
        regularizers = (tf.nn.l2_loss(W_whimh_fc1) + tf.nn.l2_loss(b_whimh_fc1) +
                        tf.nn.l2_loss(W_whimh_fc2) + tf.nn.l2_loss(b_whimh_fc2) )
                        #tf.nn.l2_loss(W_whimh_fc3) + tf.nn.l2_loss(b_whimh_fc3) )
    with tf.name_scope('loss'):
        #carefully deal overfitting
        loss=cross_entropy+regularTerm*regularizers
    with tf.name_scope('slover'):
        train_step = tf.train.AdamOptimizer(learningRate).minimize(loss)
    with tf.name_scope('measure'):
        with tf.name_scope('predict'):
            predctions=tf.argmax(y_conv,1, name='predictions_')
        with tf.name_scope('groundtruth'):
            ground_truth=tf.argmax(y_tf_2,1)
        correct_prediction = tf.equal(predctions, ground_truth)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [23]:
sess.run(tf.initialize_all_variables())

In [ ]:
k=0
for i in range(1,20000):
  arr=np.arange(len(data['X_train']))
  np.random.shuffle(arr)
  idx = arr[:30]
  train_step.run(feed_dict={x_tfed:data['X_train'][idx], y_tf: data['y_train'][idx],keep_prob:0.5,learningRate:learningObject,regularTerm:1e-3})
  
  if i%100==0:
    print("measurement")
    ############testing acc#########
    testing_accuracy = sess.run(accuracy, feed_dict={   
        x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1})
    ############training acc###########
    if i%500==0:
        train_accuracy = sess.run(accuracy, feed_dict={   
                x_tfed:data['X_train'], y_tf: data['y_train'],keep_prob:1})
        print("step %d, training accuracy %g, testing acc %g"%(i, train_accuracy,testing_accuracy))
    else:
        print("step %d, testing acc %g"%(i,testing_accuracy))
        
#overall
print("test accuracy %g"%accuracy.eval(feed_dict={
    x_tfed:data['X_val'], y_tf: data['y_val'],keep_prob:1}))

In [17]:
# close it to clear graph
tf.reset_default_graph()
sess.close()